# Imports

In [98]:
from openmm import MonteCarloMembraneBarostat, unit
from openmm.app import CharmmParameterSet, CharmmPsfFile, PME, HBonds
import os, bz2, openmm
from enhanced_sampling import system_building as sb, cv_building as cv, system_saving as ss

## Setting Variables

In [17]:
psf_path = '../systems/system00/step5_input.psf'
charmm_param_dir = "../forcefield_data/charmm-params/"
state_path = "../systems/system00/state.xml.bz2"
ref_dir = "../systems/system01/"

In [18]:
hydrogen_mass = 4.0 * unit.amu
temperature = 303.15 * unit.kelvin
friction = 1 / unit.picoseconds
time_step = 0.002 * unit.picoseconds
pressure = 1 * unit.bar
surface_tension = 0  # units are complicated

In [8]:
nonbonded_method = PME
constraints = HBonds

# Loading Files

In [9]:
psf = CharmmPsfFile(psf_path)

In [10]:
param_paths = [os.path.join(charmm_param_dir, path) for path in os.listdir(charmm_param_dir)]
params = CharmmParameterSet(*param_paths)

In [11]:
with bz2.open(state_path, 'rb') as infile:
    state = openmm.XmlSerializer.deserialize(infile.read().decode())

In [12]:
x, y, z = state.getPeriodicBoxVectors()
psf.setBox(x[0], y[1], z[2])

## Load reference

In [20]:
ref_dict = sb.load_input_dir(ref_dir, load_psf=False)
ref_positions = ref_dict['positions']

# Build System Basics

In [13]:
system = psf.createSystem(params,
                              nonbondedMethod=nonbonded_method,
                              constraints=constraints,
                              removeCMMotion=False,
                              hydrogenMass=hydrogen_mass)

integrator = openmm.LangevinMiddleIntegrator(temperature,
                                             friction,
                                             time_step)

barostat = openmm.MonteCarloMembraneBarostat(pressure,
                                             surface_tension,
                                             temperature,
                                             MonteCarloMembraneBarostat.XYIsotropic,
                                             MonteCarloMembraneBarostat.ZFree
                                             )
barostat.setFrequency(50)

system.addForce(barostat)

8

# Add Harmonic pulling forces

In [22]:
len(ref_positions)

151920

In [23]:
len(state.getPositions())

151920

In [33]:
idx_list = cv.get_openmm_idx(psf.topology, selection="protein_ca")

Using protein selection: protein_ca and res_list: False


In [40]:
restraint_positions = {idx: ref_positions[idx] for idx in idx_list}

In [42]:
restraint_positions[8]

Quantity(value=Vec3(x=4.470702410136781, y=5.5096778694211075, z=3.3690945485267463), unit=nanometer)

In [60]:
def create_harmonic_pulling_force(restraint_positions, spring_constant):
    """

    :return:
    """
    force = openmm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
    force.addGlobalParameter("k", spring_constant)
    force.addPerParticleParameter("x0")
    force.addPerParticleParameter("y0")
    force.addPerParticleParameter("z0")
    
    for atom_idx, (x,y,z) in restraint_positions.items():
        force.addParticle(atom_idx, [x, y, z])
    return force

In [72]:
force = create_harmonic_pulling_force(restraint_positions, 1000)
force.setForceGroup(20)

In [73]:
force.getNumParticles()

894

In [74]:
force.getGlobalParameterDefaultValue(0)

1000.0

In [75]:
system.addForce(force)

9

# Build Simulation Context

In [76]:
sim = openmm.app.Simulation(psf.topology,
                                system=system,
                                integrator=integrator,
                                )

## set state

In [77]:
sim.context.setState(state)

# Check values of restraint force

In [80]:
state0 = sim.context.getState(getForces=True,
                                 getEnergy=True,
                                 groups={20})

In [81]:
state0.getPotentialEnergy()

Quantity(value=163693.734375, unit=kilojoule/mole)

In [86]:
state0.getForces()[8]

Quantity(value=Vec3(x=173.9091796875, y=568.2391967773438, z=186.10000610351562), unit=kilojoule/(nanometer*mole))

## Check whole system potential energy

In [87]:
_state = sim.context.getState(getForces=True,
                                 getEnergy=True,)

In [88]:
_state.getPotentialEnergy()

Quantity(value=-1190145.6245913636, unit=kilojoule/mole)

### doesn't seem to bad, still 1 order of magnitude smaller

# Try running simulation

In [89]:
sim.step(100)

In [90]:
_state = sim.context.getState(getForces=True,
                                 getEnergy=True,)
_state.getPotentialEnergy()

Quantity(value=-1261090.6245913636, unit=kilojoule/mole)

In [91]:
state1 = sim.context.getState(getForces=True,
                                 getEnergy=True,
                             groups={20})
state1.getPotentialEnergy()

Quantity(value=78202.1875, unit=kilojoule/mole)

In [92]:
sim.step(100)

In [93]:
state1 = sim.context.getState(getForces=True,
                                 getEnergy=True,
                             groups={20})
state1.getPotentialEnergy()

Quantity(value=16902.0078125, unit=kilojoule/mole)

In [94]:
sim.step(100)

In [95]:
state1 = sim.context.getState(getForces=True,
                                 getEnergy=True,
                             groups={20})
state1.getPotentialEnergy()

Quantity(value=13406.0234375, unit=kilojoule/mole)

In [96]:
sim.step(100)
state1 = sim.context.getState(getForces=True,
                                 getEnergy=True,
                             groups={20})
state1.getPotentialEnergy()

Quantity(value=14736.69140625, unit=kilojoule/mole)

In [97]:
sim.step(100)
state1 = sim.context.getState(getForces=True,
                                 getEnergy=True,
                             groups={20})
state1.getPotentialEnergy()

Quantity(value=10300.5615234375, unit=kilojoule/mole)

In [100]:
import os

In [102]:
os.mkdir("./pulling-test")

In [103]:
ss.write_simulation_files(sim, "./pulling-test/")

Saving state
Saving cif
Saving system
Saving integrator


# Well I feel pretty dumb. that was so easy